In [1]:
from pandas.tseries.offsets import DateOffset
import ee
import yaml
import time
import os
import json
import pandas as pd
from utils import exportImageToGDrive,exportImageToGCS
from download_sen12 import *

ee.Initialize()

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
config_dict = load_config('config.yml')
source = config_dict['data_list'][0]
sensor = config_dict['sensors'][0]
# export_folder = config_dict['drive_folder']
export_folder = config_dict['bucket']

print(export_folder)

project-canopy-temp-2


C:\Users\David\canopy\canopy-gis\google-earth-explore\GEE_DataDownloader\download_sen12.py:364: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(stream)


In [3]:
start = pd.to_datetime('01-01-2020')

start

Timestamp('2020-01-01 00:00:00')

In [8]:
def import_aois(csv_loc, Full_Congo_Pull=False, start_date=None,
                end_date=None, days_duration=90, poly_start=0, poly_limit=None):
    features = []
    polygons = []
    day_offset = days_duration / 2
    start_end_list = []
    
    if Full_Congo_Pull:
        with open(csv_loc,"r",encoding='utf-8') as jsonfile:
            data = json.load(jsonfile)
            for geometry in data["features"]:
                polygon = geometry["geometry"]["coordinates"][0][0]
                poly_obj = ee.Geometry.Polygon(polygon)
                feature = ee.Feature(poly_obj)
                features.append(feature)
                
            original_date = pd.to_datetime('01-01-2020')
            start = (original_date + DateOffset(days=-30))
            end = (original_date + DateOffset(days=30))
            start_date = str(start)[:10]
            end_date = str(end)[:10]
            date_dict = {
                'start_date': start_date,
                'end_date': end_date,
                'original_date': original_date,
                'day_offset': 30,
                'area': 1
            }
            
            start_end_list = [date_dict] * len(features)
            
            return ee.FeatureCollection(features), start_end_list
            
    else:
        feature_id = poly_start
        
        if poly_limit:
            df_labels = pd.read_csv(csv_loc, skiprows=range(1, poly_start+1), nrows=poly_limit)
        else:
            df_labels = pd.read_csv(csv_loc, skiprows=range(1, poly_start+1))

        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined","tile date","area (km2)"]]
        df_labels["tile date"] = pd.to_datetime(df_labels["tile date"])
        start = (df_labels["tile date"] + DateOffset(days=-day_offset))
        end = (df_labels["tile date"] + DateOffset(days=day_offset))
        for i in range(len(start)):
            start_date = str(start[i])[:10]
            end_date = str(end[i])[:10]
            original_date = df_labels.loc[i, 'tile date']
            area = df_labels.loc[i, 'area (km2)']
            date_dict = {
                'start_date': start_date,
                'end_date': end_date,
                'original_date': original_date,
                'day_offset': day_offset,
                'area': area
            }
            start_end_list.append(date_dict)

        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])
            
        for poly in polygons:
            # create an roi. first item in Misha's label list
            feature_id += 1 
            # create geometry object, create feature object, append to features list for feature collection creation 
            polys = ee.Geometry.Polygon(poly)
            feature = ee.Feature(polys,{"name":feature_id})
            features.append(feature)
            
        return ee.FeatureCollection(features),start_end_list,df_labels


# ### CHANGE BELOW PATH ###
zhenya_label_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv"
david__label_path = 'D:/canopy_data/csvs/polygons_101320.csv'
zhenya_tilelist_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/tile_lists/shub_tile_list/tiles_polygons_full.csv"
david_grid_path = 'D:/canopy_data/Geometry/Congo_Basin_Shapefile_Grid_v1.geojson'

In [10]:
fc,date_range_list = import_aois(david_grid_path, Full_Congo_Pull=True)

In [11]:
date_range_list[0:5]

[{'start_date': '2019-12-02',
  'end_date': '2020-01-31',
  'original_date': Timestamp('2020-01-01 00:00:00'),
  'day_offset': 30,
  'area': 1},
 {'start_date': '2019-12-02',
  'end_date': '2020-01-31',
  'original_date': Timestamp('2020-01-01 00:00:00'),
  'day_offset': 30,
  'area': 1},
 {'start_date': '2019-12-02',
  'end_date': '2020-01-31',
  'original_date': Timestamp('2020-01-01 00:00:00'),
  'day_offset': 30,
  'area': 1},
 {'start_date': '2019-12-02',
  'end_date': '2020-01-31',
  'original_date': Timestamp('2020-01-01 00:00:00'),
  'day_offset': 30,
  'area': 1},
 {'start_date': '2019-12-02',
  'end_date': '2020-01-31',
  'original_date': Timestamp('2020-01-01 00:00:00'),
  'day_offset': 30,
  'area': 1}]

In [12]:
fc

In [13]:
fc.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[7.774437227891959, 5.590436336],
    [7.789316305, 5.590436336],
    [7.789316305, 5.607344378168228],
    [7.774437227891959, 5.590436336]]]},
 'id': '0',
 'properties': {}}